In [1]:
import clip
import torch
import numpy as np
import sklearn
import transformers
from tqdm import tqdm
import PIL

In [2]:
# # works only with old version of transformers
# !pip install -U transformers==4.37.1

In [3]:
# # there is a bug in ipykernel that prevents downloading of BAAI/bge-large-en-v1.5, so we need to be sure that it is updated
# # https://github.com/huggingface/xet-core/issues/526
# !pip install -U ipykernel>=7.1.0

# Calculate Metrics for UForm Model

In [4]:
def get_metrics_txt_emb(text):
    METRICS_TXT_EMB_MODEL_NAME = "BAAI/bge-large-en-v1.5"

    tokenizer = transformers.AutoTokenizer.from_pretrained(METRICS_TXT_EMB_MODEL_NAME)
    model = transformers.AutoModel.from_pretrained(METRICS_TXT_EMB_MODEL_NAME)
    model.eval()
    
    encoded_input = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]

    return torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)[0].numpy()

## Load Data

In [5]:
def load_image_markup(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        while True:
            line = f.readline()
            if len(line) == 0:
                break
            sep_ind = line.find("\"")
            path = ("../" + line[:sep_ind]).strip()
            desc = line[sep_ind + 1:].strip()[:-1]
            data.append((path, desc))
    return data


img_markup = load_image_markup("../data/matching_images.txt")

## Metrics

In [6]:
def calc_metrics(markup, model, processor_obj):
    PROMPT = "List the objects in the photo in one sentence, no information about background needed"

    text_list = [markup_line[1].lower() for markup_line in markup]
    # some descriptions are the same or mostly same (start from the same words)
    unique_indices = []
    for idx in range(len(text_list)):
        if idx == 0:
            unique_indices.append(idx)
            continue
        prev_started = any(txt.startswith(text_list[idx]) for txt in text_list[:idx])
        current_started = any(text_list[idx].startswith(txt) for txt in text_list[:idx])
        if prev_started or current_started:
            continue
        unique_indices.append(idx)
    unique_text_list = [text_list[idx] for idx in unique_indices]
    emb_unique_text_list = [get_metrics_txt_emb(txt) for txt in unique_text_list]

    y_true_list = []
    distance_list = []
    for img_idx, markup_line in tqdm(enumerate(markup), total=len(text_list)):
        inputs = processor_obj(text=[PROMPT], images=[PIL.Image.open(markup_line[0])], return_tensors="pt")
        with torch.inference_mode():
             output = model.generate(
                **inputs,
                do_sample=False,
                use_cache=True,
                max_new_tokens=256,
                eos_token_id=151645,
                pad_token_id=processor_obj.tokenizer.pad_token_id
                
            )

        prompt_len = inputs["input_ids"].shape[1]
        decoded_out = processor_obj.batch_decode(output[:, prompt_len:], skip_special_tokens=True)[0]

        emb_decoded_out = get_metrics_txt_emb(decoded_out)
        distance_per_image = [np.linalg.norm(emb_unique_text - emb_decoded_out) for emb_unique_text in emb_unique_text_list]

        if img_idx not in unique_indices:
            assert markup_line[1].lower() == text_list[img_idx]
            txt_to_find = markup_line[1].lower()
            u_img_idx = -1
            for i in range(len(unique_text_list)):
                if unique_text_list[i].startswith(txt_to_find) or txt_to_find.startswith(unique_text_list[i]):
                    u_img_idx = i
                    break
            assert u_img_idx >= 0
            assert u_img_idx < img_idx
        else:
            u_img_idx = unique_indices.index(img_idx)

        y_true_list += [1 if idx == u_img_idx else 0 for idx in range(len(unique_text_list))]
        distance_list += distance_per_image

    y_true_vec = np.array(y_true_list)
    max_distance = max(distance_list)
    logit_vec = np.array([(max_distance - dist)/max_distance for dist in distance_list])
    assert y_true_vec.shape == logit_vec.shape

    assert logit_vec.min() >= 0
    assert logit_vec.max() <= 1
    assert np.allclose(np.sort(np.unique(y_true_vec)), np.array([0, 1]))

    # https://stats.stackexchange.com/q/287117/
    prevalence = np.count_nonzero(y_true_vec == 1, keepdims=False)/len(y_true_vec)
    assert prevalence > 0
    fpr_vec, tpr_vec, thr_vec = sklearn.metrics.roc_curve(y_true_vec, logit_vec)
    recall_vec = tpr_vec
    tnr_vec = 1 - fpr_vec

    zero_div_idxs = np.where((recall_vec*prevalence) + ((1 - tnr_vec)*(1 - prevalence)) == 0)[0]
    if zero_div_idxs.size > 0:
        # to avoid zero-division warning from numpy below
        recall_vec[zero_div_idxs] += 1e-8
    precision_vec = (recall_vec*prevalence)/((recall_vec*prevalence) + ((1 - tnr_vec)*(1 - prevalence)))

    zero_div_idxs = np.where(precision_vec + recall_vec == 0)[0]
    if zero_div_idxs.size > 0:
        # to avoid zero-division warning from numpy below
        precision_vec[zero_div_idxs] += 1e-8
        recall_vec[zero_div_idxs] += 1e-8
    f1_vec = 2*(precision_vec*recall_vec)/(precision_vec + recall_vec)

    opt_thr = thr_vec[np.argmax(f1_vec)]
    if np.isinf(opt_thr):
        opt_thr = 1

    # using ">=" below instead of ">" is extremely important, because sklearn cn return edge values for threshold
    tp = np.count_nonzero((logit_vec >= opt_thr) & (y_true_vec == 1))
    fp = np.count_nonzero((logit_vec >= opt_thr) & (y_true_vec == 0))
    tn = np.count_nonzero((logit_vec < opt_thr) & (y_true_vec == 0))
    fn = np.count_nonzero((logit_vec < opt_thr) & (y_true_vec == 1))

    tp_list = [
        (distance_list[tp_idx], text_list[tp_idx % len(unique_text_list)])
        for tp_idx in np.nonzero((logit_vec >= opt_thr) & (y_true_vec == 1))[0]
    ]
    tp_list = [(sum(dist for dist, txt in tp_list if txt == u_txt), u_txt) for u_txt in set([txt for dist, txt in tp_list])]
    tp_list.sort(key=lambda item: item[0])
    fp_list = [
        (distance_list[fp_idx], text_list[fp_idx % len(unique_text_list)])
        for fp_idx in np.nonzero((logit_vec >= opt_thr) & (y_true_vec == 0))[0]
    ]
    fp_list = [(sum(dist for dist, txt in fp_list if txt == u_txt), u_txt) for u_txt in set([txt for dist, txt in fp_list])]
    fp_list.sort(key=lambda item: item[0])
    opt_thr = max_distance - (float(opt_thr)*max_distance)

    return tp, fp, tn, fn, opt_thr, tp_list, fp_list

In [7]:
MODEL_NAME = "unum-cloud/uform-gen2-qwen-500m"
model = transformers.AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)
processor_func = transformers.AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
tp, fp, tn, fn, thr, tp_list, fp_list = calc_metrics(img_markup, model, processor_func)

if 2*tp + fp + fn > 0:
    f1 = 2*tp/(2*tp + fp + fn)
else:
    f1 = 0

if tp + fp + tn + fn > 0:
    acc = (tp + tn)/(tp + fp + tn + fn)
else:
    acc = 0

print("TRUE POSITIVES:\n\t" + "\n\t".join(f"({dist:.1f}) {txt}" for dist, txt in tp_list) + "\n\n")
print("FALSE POSITIVES:\n\t" + "\n\t".join(f"({dist:.1f}) {txt}" for dist, txt in fp_list) + "\n\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [55:57<00:00, 10.80s/it]

TRUE POSITIVES:
	(0.6) black sleeveless dress for girls
	(0.6) women's warm down jacket with zip and hood
	(0.6) dark blue women's dress
	(0.7) women's high black suede boots, women's high beige suede boots
	(0.7) warm zipped jacket with a hood
	(0.7) crib bumpers with bunnies
	(0.7) cover for baby's changing table
	(0.7) 3 doors wooden cabinet, armchair, wooden chairs
	(0.7) black kitten sits on the wooden boards
	(0.7) party dress for a girl
	(0.7) individually wrapped tea bags
	(0.7) abstract patterned floor rug
	(0.7) women's warm down jacket
	(0.7) pink rubber boots for girls with unicorns, black patent leather shoes for girls, pink summer sandals for girls, burgundy summer shoes
	(0.7) inflatable swimming ring clasp
	(0.7) grey kitten sits on the wooden boards
	(0.7) gray windbreaker with a zipper
	(0.7) girls clothing set
	(0.8) feeding baby chair
	(0.8) women's leather low boots with buckles view from the top
	(0.8) glassware, shot glasses, goblets, wine glasses, plates and a m

In [9]:
print(f"Model: {MODEL_NAME}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Threshold: {thr}")
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

Model: unum-cloud/uform-gen2-qwen-500m
TP: 78, FP: 139, TN: 76056, FN: 233
Threshold: 0.8079373836517334
Accuracy: 0.9951376362638225
F1: 0.29545454545454547
